# StyleGrid I

Extending the [DataGrid](./DataGrid.ipynb), `StyleGrid` adds support for top-level styles of the rendered grid. Advanced customization is possible with [custom renderers](./StyleGrid%20II.ipynb).

In [ ]:
if __name__ == "__main__":
    %pip install -q wxyz-notebooks

In [ ]:
from wxyz.datagrid import StyleGrid, GridStyle

In [ ]:
import pandas as pd, traitlets as T, ipywidgets as W, IPython.display as D
import importnb
from wxyz.html import AlphaColorPicker 
from wxyz.lab import DockBox, DockPop

In [ ]:
with importnb.Notebook():
    try:
        from DataGrid import make_grid
    except:
        from .DataGrid import make_grid

## GridStyle

In [ ]:
def make_a_grid_styler(style=None, grid=None, pattern='color', default_color="rgba(0,0,0,0.0)"):
    special = ['columnBackgroundColor', 'rowBackgroundColor']

    style = style or GridStyle()
    colors = [
        AlphaColorPicker(description=n, value=default_color) 
        for n in style.trait_names() 
        if pattern in n.lower()
        and n not in special
    ]
    
    box = W.VBox(colors, _dom_classes=["my-style"], layout=dict(display="flex", flex_flow="row wrap"))
    box.children += (W.HTML("""<style>
        .my-style .widget-colorpicker {
            display: flex;
            flex-direction: column;
            height: 4em;
        }
        .my-style .widget-colorpicker label {
            display: flex;
            width: 100%;
        }
    </style>"""),)
    
    if grid:
        [T.dlink((c, 'value'), (grid.grid_style, c.description)) for c in colors]
    
    return box

In [ ]:
if __name__ == "__main__":
    styler = make_a_grid_styler()
    D.display(styler)

## Make a Grid

In [ ]:
_default_container = lambda *args, **kwargs: [
    kwargs.update(layout=dict(height="60vh"), dock_layout={'type': 'split-area',
 'orientation': 'horizontal',
 'children': [{'type': 'tab-area', 'widgets': [0], 'currentIndex': 0},
  {'type': 'tab-area', 'widgets': [1], 'currentIndex': 0}],
 'sizes': [8, 3]}),
    DockBox(*args, **kwargs)
][-1]

def make_style_grid_i(GridClass=StyleGrid, df=None, container=None):
    container = container or _default_container
    pg = make_grid(GridClass, df=df)
    dg = pg.dg
    st = make_a_grid_styler(grid=dg)

    rs = W.IntSlider(description='row height')
    cs = W.IntSlider(description='column width')
    hv = W.SelectionSlider(description="headers?", default_value="all", options=["all", "row", "column", "none"])
    hr = W.HTML()
    hc = W.HTML()

    W.jslink((dg, "row_size"), (rs, "value"))
    W.jslink((dg, "column_size"), (cs, "value"))
    W.link((dg, "header_visibility"), (hv, "value"))
    grid_playground = container([
        W.VBox([
            pg.children[1], 
        ], layout=dict(flex="1")),
        W.VBox([*pg.children[0].children, hr, hc, cs, rs, hv, *st.children]), 
    ])
    grid_playground.dg = dg
    return grid_playground

In [ ]:
if __name__ == "__main__":
    pg = make_style_grid_i()
    display(pg)

## Make a color scheme

In [ ]:
def make_a_color_schemer():
    o = dict(orientation="vertical")
    steps = W.IntSlider(value=5, min=0, max=100, description="steps", **o)
    colors = r, g, b, R, G, B = [W.IntSlider(value=0, min=0, max=255, description=d, **o) for d in "rgbRGB"]
    alphas = a, A = [W.FloatSlider(value=0, description=d, min=0.0, max=1.0, **o) for d in "aA"]
    sliders = steps, *colors, *alphas
    h = W.HTML()
    box = W.HBox([*sliders])
    box.add_traits(value=T.Tuple())
    T.dlink((box, "value"), (h, "value"), str)
    
    def _update(*_):
        s = steps.value
        if s == 0:
            box.value = [
            ]
            return
        
        value = [
            [r.value, g.value, b.value, a.value]
        ]
                
        if s > 2:
            for i in range(1, s):
                value += [[
                    int(r.value + ((R.value - r.value) / s) * i),
                    int(g.value + ((G.value - g.value) / s) * i),
                    int(b.value + ((B.value - b.value) / s) * i),
                    a.value + ((A.value - a.value) / s) * i,
                ]]
        
        if steps.value > 1:
            value += [
                [R.value, G.value, B.value, A.value]
            ]

        box.value = [
            "rgba({}, {}, {}, {})".format(*v) for v in value
        ]
        
    setattr(box, "_update", _update)
    
    [s.observe(_update) for s in sliders]

    return box

In [ ]:
def make_style_grid_i_remix(GridClass=StyleGrid, df=None, container=None):
    pg = make_style_grid_i(GridClass, df, container)
    row_background_color = make_a_color_schemer()
    column_background_color = make_a_color_schemer()
    T.dlink((row_background_color, "value"), (pg.dg.grid_style, "rowBackgroundColor"))
    T.dlink((column_background_color, "value"), (pg.dg.grid_style, "columnBackgroundColor"))
    dynamics = W.HBox([row_background_color, column_background_color])
    pg.children += dynamics,
    dynamics.description = "row and column color"
    pg.dock_layout = {'type': 'split-area',
     'orientation': 'horizontal',
     'children': [{'type': 'tab-area', 'widgets': [0], 'currentIndex': 0},
      {'type': 'split-area',
       'orientation': 'vertical',
       'children': [{'type': 'tab-area', 'widgets': [1], 'currentIndex': 0},
        {'type': 'tab-area', 'widgets': [2], 'currentIndex': 0}],
       'sizes': [1, 1]}],
     'sizes': [6, 4]}
    row_background_color._update()
    column_background_color._update()
    return pg

In [ ]:
if __name__ == "__main__":
    pg2 = make_style_grid_i_remix()
    D.display(pg2)

In [ ]:
if __name__ == "__main__":
    with __import__("importnb").Notebook():
        from wxyz.notebooks import Utils
        Utils.maybe_log_widget_counts()